In [282]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

#Modeling
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedKFold
import lightgbm as lgbm
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score      
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.linear_model import ElasticNet, BayesianRidge
from sklearn.ensemble import AdaBoostRegressor




from catboost import CatBoostRegressor

#from sklego.linear_model import LADRegression
from xgboost import XGBRegressor
from scipy.stats import yeojohnson
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from mlxtend.regressor import StackingCVRegressor
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings('ignore')

In [283]:
import pandas as pd

# Definisikan nama file
filename = "booking_dataset_2024_06_03_with_nan.csv"

# Membaca dataset dari file CSV
raw_data = pd.read_csv(filename)

# Menampilkan 5 baris pertama dari dataset
raw_data.head()

,booking_id,booking_status,booking_check_in,booking_check_out,booking_guest_number,booking_currency,booking_received_timestamp,listing_id,review_sentiment,review_sentiment_score,...,workspace,pool,parking,gym,kitchen,property_id1,area_id,area_name,area_distance_to_airport,airport_pickup_price_idr
0,2REFMR,1,2016-04-06,2016-04-16,NaN,2,NaN,6384209,positive,0.655466,...,1.0,1.0,1.0,0.0,1.0,PR0196,A0012,Nusa Dua,45 minutes,325000.0
1,9NBTPK,1,2016-04-02,2016-04-05,NaN,2,NaN,945876,positive,0.656976,...,1.0,1.0,1.0,0.0,1.0,PR0242,A0012,Nusa Dua,45 minutes,325000.0
2,A2SQJN,1,2016-10-06,2016-10-08,NaN,2,NaN,945876,positive,0.687372,...,1.0,1.0,1.0,0.0,1.0,PR0242,A0012,Nusa Dua,45 minutes,325000.0
3,AKJCTJ,1,2015-10-30,2015-11-04,NaN,2,NaN,6384209,positive,0.979358,...,1.0,1.0,1.0,0.0,1.0,PR0196,A0012,Nusa Dua,45 minutes,325000.0
4,B3T92M,1,2016-10-10,2016-10-14,NaN,2,NaN,6384209,positive,0.670770,...,1.0,1.0,1.0,0.0,1.0,PR0196,A0012,Nusa Dua,45 minutes,325000.0


In [284]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56825 entries, 0 to 56824
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   booking_id                  56825 non-null  object 
 1   booking_status              56825 non-null  int64  
 2   booking_check_in            56825 non-null  object 
 3   booking_check_out           56825 non-null  object 
 4   booking_guest_number        9015 non-null   float64
 5   booking_currency            56825 non-null  int64  
 6   booking_received_timestamp  30445 non-null  object 
 7   listing_id                  56825 non-null  object 
 8   review_sentiment            6030 non-null   object 
 9   review_sentiment_score      6030 non-null   float64
 10  cancellation_policy         0 non-null      float64
 11  booking_earned_in_idr       51166 non-null  float64
 12  length_of_stay              56825 non-null  int64  
 13  booking_window              304

In [285]:
raw_data.describe()

,booking_status,booking_guest_number,booking_currency,review_sentiment_score,cancellation_policy,booking_earned_in_idr,length_of_stay,booking_window,seasonality,average_daily_rate,...,wifi,tv,cable_tv,ac,workspace,pool,parking,gym,kitchen,airport_pickup_price_idr
count,56825.000000,9015.000000,56825.000000,6030.000000,0.0,5.116600e+04,56825.000000,30445.000000,56825.000000,5.116600e+04,...,42082.000000,42082.000000,42082.000000,42082.000000,42082.000000,42082.000000,42082.000000,42082.000000,42082.000000,48044.000000
mean,1.063968,2.618525,1.233295,0.729690,NaN,2.578506e+06,2.839366,23.068944,0.372107,8.505980e+05,...,0.989449,0.576137,0.213892,0.991754,0.936101,0.820755,0.947032,0.005988,0.321468,325682.499376
std,0.281550,2.274074,0.427279,0.413806,NaN,4.703238e+06,2.982119,36.979856,0.483371,1.018562e+06,...,0.102175,0.494175,0.410056,0.090432,0.244576,0.383562,0.223972,0.077153,0.467045,39039.625935
min,1.000000,0.000000,1.000000,-0.998203,NaN,0.000000e+00,-30.000000,0.000000,0.000000,-4.938270e+04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,195000.000000
25%,1.000000,2.000000,1.000000,0.704758,NaN,6.367480e+05,1.000000,1.000000,0.000000,3.492000e+05,...,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,325000.000000
50%,1.000000,2.000000,1.000000,0.887651,NaN,1.236796e+06,2.000000,6.000000,0.000000,5.130320e+05,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,325000.000000
75%,1.000000,2.000000,1.000000,0.972097,NaN,2.520008e+06,3.000000,31.000000,1.000000,9.409000e+05,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,325000.000000
max,6.000000,30.000000,3.000000,0.999999,NaN,1.303185e+08,349.000000,383.000000,1.000000,2.129842e+07,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,585000.000000


In [286]:
raw_data.isnull().sum()

booking_id                        0
booking_status                    0
booking_check_in                  0
booking_check_out                 0
booking_guest_number          47810
booking_currency                  0
booking_received_timestamp    26380
listing_id                        0
review_sentiment              50795
review_sentiment_score        50795
cancellation_policy           56825
booking_earned_in_idr          5659
length_of_stay                    0
booking_window                26380
seasonality                       0
average_daily_rate             5659
unit_id                          14
status                          447
bedroom                         513
bathroom                      11595
beds                          11595
capacity                      15556
property_id                     447
avg_daily_rate_per_unit        5659
wifi                          14743
tv                            14743
cable_tv                      14743
ac                          

In [287]:

raw_data['booking_check_in'] = pd.to_datetime(raw_data['booking_check_in'])
raw_data['booking_check_out'] = pd.to_datetime(raw_data['booking_check_out'])

def convert_to_minutes(time_str):
    if pd.isna(time_str):
        return None
    if 'hour' in time_str:
        hours = int(time_str.split()[0])
        return hours * 60
    elif 'minute' in time_str:
        minutes = int(time_str.split()[0])
        return minutes
    else:
        return None

raw_data['area_distance_to_airport'] = raw_data['area_distance_to_airport'].apply(convert_to_minutes)

raw_data['booking_check_in_date'] = raw_data['booking_check_in'].dt.day
raw_data['booking_check_in_month'] = raw_data['booking_check_in'].dt.month
raw_data['booking_check_in_year'] = raw_data['booking_check_in'].dt.year

raw_data['booking_check_out_date'] = raw_data['booking_check_out'].dt.day
raw_data['booking_check_out_month'] = raw_data['booking_check_out'].dt.month
raw_data['booking_check_out_year'] = raw_data['booking_check_out'].dt.year



In [288]:
raw_data = raw_data.drop(['booking_id','booking_check_in','booking_check_out','booking_received_timestamp','cancellation_policy','status','booking_window'
], axis = 1)


In [289]:
raw_data.isnull().sum()

booking_status                  0
booking_guest_number        47810
booking_currency                0
listing_id                      0
review_sentiment            50795
review_sentiment_score      50795
booking_earned_in_idr        5659
length_of_stay                  0
seasonality                     0
average_daily_rate           5659
unit_id                        14
bedroom                       513
bathroom                    11595
beds                        11595
capacity                    15556
property_id                   447
avg_daily_rate_per_unit      5659
wifi                        14743
tv                          14743
cable_tv                    14743
ac                          14743
workspace                   14743
pool                        14743
parking                     14743
gym                         14743
kitchen                     14743
property_id1                  578
area_id                       578
area_name                     578
area_distance_

In [290]:
from sklearn.preprocessing import LabelEncoder

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Menggunakan LabelEncoder untuk kolom unit_id
raw_data['listing_id'] = label_encoder.fit_transform(raw_data['listing_id'])

raw_data['review_sentiment'] = label_encoder.fit_transform(raw_data['review_sentiment'])

raw_data['property_id1'] = label_encoder.fit_transform(raw_data['property_id1'])


raw_data['unit_id'] = label_encoder.fit_transform(raw_data['unit_id'])

raw_data['unit_id'] = label_encoder.fit_transform(raw_data['unit_id'])

raw_data['property_id'] = label_encoder.fit_transform(raw_data['property_id'])

raw_data['area_id'] = label_encoder.fit_transform(raw_data['area_id'])

raw_data['area_name'] = label_encoder.fit_transform(raw_data['area_name'])

print(raw_data[['unit_id', 'review_sentiment','listing_id','property_id', 'area_id', 'area_name']].head())


   unit_id  review_sentiment  listing_id  property_id  area_id  area_name
0      335                 2        1817          154       11         13
1      334                 2        1893          192       11         13
2      334                 2        1893          192       11         13
3      335                 2        1817          154       11         13
4      335                 2        1817          154       11         13


In [291]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# Memilih kolom yang akan digunakan untuk clustering
columns_for_clustering = ['bedroom', 'bathroom', 'beds', 'capacity','wifi','tv','cable_tv','ac','workspace','pool',
                          'booking_guest_number','parking','gym','kitchen']

# Mengisi nilai yang hilang sementara dengan SimpleImputer (mean strategy)
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(raw_data[columns_for_clustering])

# Normalisasi data sebelum clustering
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

# Menentukan jumlah cluster (misalnya, 5)
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(data_scaled)

# Menambahkan kolom cluster ke DataFrame asli
raw_data['cluster'] = clusters

# Mengisi nilai yang hilang dengan centroid dari masing-masing cluster
for column in columns_for_clustering:
    for cluster in range(5):
        cluster_mean = raw_data[raw_data['cluster'] == cluster][column].mean()
        raw_data.loc[(raw_data[column].isnull()) & (raw_data['cluster'] == cluster), column] = cluster_mean

# Mengecek kembali nilai yang hilang setelah imputasi
print("\nJumlah nilai yang hilang setelah imputasi:\n", raw_data.isnull().sum())

# Menghapus kolom cluster karena sudah tidak diperlukan
raw_data.drop(columns=['cluster'], inplace=True)


Jumlah nilai yang hilang setelah imputasi:
 booking_status                  0
booking_guest_number            0
booking_currency                0
listing_id                      0
review_sentiment                0
review_sentiment_score      50795
booking_earned_in_idr        5659
length_of_stay                  0
seasonality                     0
average_daily_rate           5659
unit_id                         0
bedroom                         0
bathroom                        0
beds                            0
capacity                        0
property_id                     0
avg_daily_rate_per_unit      5659
wifi                            0
tv                              0
cable_tv                        0
ac                              0
workspace                       0
pool                            0
parking                         0
gym                             0
kitchen                         0
property_id1                    0
area_id                         0
are

In [292]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# Memilih kolom yang akan digunakan untuk clustering
columns_for_clustering = ['booking_check_in_date','booking_check_in_month','booking_check_in_year',
                          'booking_check_in_year','booking_check_out_month','booking_check_out_year']

# Mengisi nilai yang hilang sementara dengan SimpleImputer (mean strategy)
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(raw_data[columns_for_clustering])

# Normalisasi data sebelum clustering
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)

# Menentukan jumlah cluster (misalnya, 5)
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(data_scaled)

# Menambahkan kolom cluster ke DataFrame asli
raw_data['cluster'] = clusters

# Mengisi nilai yang hilang dengan centroid dari masing-masing cluster
for column in columns_for_clustering:
    for cluster in range(5):
        cluster_mean = raw_data[raw_data['cluster'] == cluster][column].mean()
        raw_data.loc[(raw_data[column].isnull()) & (raw_data['cluster'] == cluster), column] = cluster_mean

# Mengecek kembali nilai yang hilang setelah imputasi
print("\nJumlah nilai yang hilang setelah imputasi:\n", raw_data.isnull().sum())

# Menghapus kolom cluster karena sudah tidak diperlukan
raw_data.drop(columns=['cluster'], inplace=True)


Jumlah nilai yang hilang setelah imputasi:
 booking_status                  0
booking_guest_number            0
booking_currency                0
listing_id                      0
review_sentiment                0
review_sentiment_score      50795
booking_earned_in_idr        5659
length_of_stay                  0
seasonality                     0
average_daily_rate           5659
unit_id                         0
bedroom                         0
bathroom                        0
beds                            0
capacity                        0
property_id                     0
avg_daily_rate_per_unit      5659
wifi                            0
tv                              0
cable_tv                        0
ac                              0
workspace                       0
pool                            0
parking                         0
gym                             0
kitchen                         0
property_id1                    0
area_id                         0
are

In [293]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
columns_to_impute = [ 'airport_pickup_price_idr','area_distance_to_airport','review_sentiment_score']
imputer = IterativeImputer(max_iter=10, random_state=0)

# Memilih subset data untuk kolom-kolom yang akan diimputasi
data_to_impute = raw_data[columns_to_impute]

# Melakukan imputasi
data_imputed = imputer.fit_transform(data_to_impute)

# Mengganti nilai yang telah diimputasi kembali ke dataset asli
raw_data[columns_to_impute] = data_imputed

# Menampilkan hasil imputasi
print("Hasil imputasi untuk kolom area_distance_to_airport dan airport_pickup_price_idr:")
print(raw_data[columns_to_impute].head())
raw_data.isnull().sum()

Hasil imputasi untuk kolom area_distance_to_airport dan airport_pickup_price_idr:
   airport_pickup_price_idr  area_distance_to_airport  review_sentiment_score
0                  325000.0                      45.0                0.655466
1                  325000.0                      45.0                0.656976
2                  325000.0                      45.0                0.687372
3                  325000.0                      45.0                0.979358
4                  325000.0                      45.0                0.670770


booking_status                 0
booking_guest_number           0
booking_currency               0
listing_id                     0
review_sentiment               0
review_sentiment_score         0
booking_earned_in_idr       5659
length_of_stay                 0
seasonality                    0
average_daily_rate          5659
unit_id                        0
bedroom                        0
bathroom                       0
beds                           0
capacity                       0
property_id                    0
avg_daily_rate_per_unit     5659
wifi                           0
tv                             0
cable_tv                       0
ac                             0
workspace                      0
pool                           0
parking                        0
gym                            0
kitchen                        0
property_id1                   0
area_id                        0
area_name                      0
area_distance_to_airport       0
airport_pi

In [294]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
columns_to_impute = [ 'booking_earned_in_idr','average_daily_rate','avg_daily_rate_per_unit']
imputer = IterativeImputer(max_iter=10, random_state=0)

# Memilih subset data untuk kolom-kolom yang akan diimputasi
data_to_impute = raw_data[columns_to_impute]

# Melakukan imputasi
data_imputed = imputer.fit_transform(data_to_impute)

# Mengganti nilai yang telah diimputasi kembali ke dataset asli
raw_data[columns_to_impute] = data_imputed

# Menampilkan hasil imputasi
print("Hasil imputasi untuk kolom area_distance_to_airport dan airport_pickup_price_idr:")
print(raw_data[columns_to_impute].head())
raw_data.isnull().sum()

Hasil imputasi untuk kolom area_distance_to_airport dan airport_pickup_price_idr:
   booking_earned_in_idr  average_daily_rate  avg_daily_rate_per_unit
0            33785261.68        3.378526e+06             5.126228e+06
1            12943833.32        4.314611e+06             4.843822e+06
2            11364687.81        5.682344e+06             4.843822e+06
3            29848669.00        5.969734e+06             5.126228e+06
4            25526762.17        6.381691e+06             5.126228e+06


booking_status              0
booking_guest_number        0
booking_currency            0
listing_id                  0
review_sentiment            0
review_sentiment_score      0
booking_earned_in_idr       0
length_of_stay              0
seasonality                 0
average_daily_rate          0
unit_id                     0
bedroom                     0
bathroom                    0
beds                        0
capacity                    0
property_id                 0
avg_daily_rate_per_unit     0
wifi                        0
tv                          0
cable_tv                    0
ac                          0
workspace                   0
pool                        0
parking                     0
gym                         0
kitchen                     0
property_id1                0
area_id                     0
area_name                   0
area_distance_to_airport    0
airport_pickup_price_idr    0
booking_check_in_date       0
booking_check_in_month      0
booking_ch

In [295]:
data = raw_data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56825 entries, 0 to 56824
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   booking_status            56825 non-null  int64  
 1   booking_guest_number      56825 non-null  float64
 2   booking_currency          56825 non-null  int64  
 3   listing_id                56825 non-null  int32  
 4   review_sentiment          56825 non-null  int32  
 5   review_sentiment_score    56825 non-null  float64
 6   booking_earned_in_idr     56825 non-null  float64
 7   length_of_stay            56825 non-null  int64  
 8   seasonality               56825 non-null  int64  
 9   average_daily_rate        56825 non-null  float64
 10  unit_id                   56825 non-null  int64  
 11  bedroom                   56825 non-null  float64
 12  bathroom                  56825 non-null  float64
 13  beds                      56825 non-null  float64
 14  capaci

In [296]:
data.isnull().sum()

booking_status              0
booking_guest_number        0
booking_currency            0
listing_id                  0
review_sentiment            0
review_sentiment_score      0
booking_earned_in_idr       0
length_of_stay              0
seasonality                 0
average_daily_rate          0
unit_id                     0
bedroom                     0
bathroom                    0
beds                        0
capacity                    0
property_id                 0
avg_daily_rate_per_unit     0
wifi                        0
tv                          0
cable_tv                    0
ac                          0
workspace                   0
pool                        0
parking                     0
gym                         0
kitchen                     0
property_id1                0
area_id                     0
area_name                   0
area_distance_to_airport    0
airport_pickup_price_idr    0
booking_check_in_date       0
booking_check_in_month      0
booking_ch

In [297]:
from sklearn.model_selection import train_test_split

# Memisahkan fitur-fitur dan target
X = data.drop(columns=['booking_earned_in_idr'])
y = data['booking_earned_in_idr']

# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Menampilkan dimensi dari data latih dan data uji
print("Dimensi data latih:", X_train.shape, y_train.shape)
print("Dimensi data uji:", X_test.shape, y_test.shape)

Dimensi data latih: (45460, 36) (45460,)
Dimensi data uji: (11365, 36) (11365,)


In [298]:
y = data['booking_earned_in_idr']
x = data.drop('booking_earned_in_idr', axis=1)

In [299]:
# apply transformation since it is skewed as we saw in the eda phase
data['booking_earned_in_idr'] = np.log1p(data['booking_earned_in_idr'])

In [300]:
skew_check = x.apply(lambda x: abs(skew(x)))
skew_check = skew_check.reset_index(name='skew')
skewness = skew_check[skew_check['skew']>0.5]
skewed_feats = skewness['index'][:-1]
skewness

,index,skew
0,booking_status,5.886724
1,booking_guest_number,6.371349
2,booking_currency,1.332097
4,review_sentiment,4.094981
5,review_sentiment_score,2.933164
6,length_of_stay,33.478140
7,seasonality,0.529174
8,average_daily_rate,4.178856
9,unit_id,0.585091
10,bedroom,2.221732


In [301]:
from sklearn.preprocessing import MinMaxScaler
# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Terapkan normalisasi Min-Max pada fitur numerik
x_scaled = pd.DataFrame(scaler.fit_transform(x), columns=x.columns)

# Gabungkan kembali fitur yang sudah dinormalisasi dengan label
data_normalized = pd.concat([x_scaled, y.reset_index(drop=True)], axis=1)

In [302]:
# apply polynomial since the features are not very linear with the label, and set the degree to 2 to prevent overfitting
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(x)

In [303]:
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import RepeatedKFold
import pandas as pd

class Model:
    def __init__(self, x, y, n_splits, repeated, rdm_seed, model, params):
        self.x = x
        self.y = y
        self.n_splits = n_splits
        self.repeated = repeated
        self.rdm_seed = rdm_seed
        self.model = model
        self.params = params
        self.cv_splitter = None
        self.best_model = None
        self.best_rmse = float('inf')
        self.best_mae = float('inf')  # Initialize best_mae attribute

    def split_data(self):
        cv = RepeatedKFold(n_splits=self.n_splits, n_repeats=self.repeated, random_state=self.rdm_seed)
        for fold, (train_idx, val_idx) in enumerate(cv.split(self.x, self.y)):
            X_train = self.x[train_idx]
            X_val = self.x[val_idx]
            y_train = self.y[train_idx]
            y_val = self.y[val_idx]
            yield X_train, X_val, y_train, y_val

    def fit(self, key):
        mae_per_fold = []
        r2_per_fold = []
        self.cv_splitter = self.split_data()

        for fold, (X_train, X_val, y_train, y_val) in enumerate(self.cv_splitter):
            model = self.model(**self.params)  # Define model outside if-elif block
            if key == 'xgb':
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='mae', early_stopping_rounds=100, verbose=False)
            elif key == 'adaboost':
                model = self._fit_adaboost(X_train, y_train)
            elif key == 'lgb':
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='mae')
            elif key == 'cat':
                model = CatBoostRegressor(**self.params, loss_function='MAE')
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
            elif key == 'bayesian_ridge':
                model = BayesianRidge(**self.params)
                model.fit(X_train, y_train)
            elif key in ['linear', 'ridge', 'lasso', 'svr']:
                model.fit(X_train, y_train)

            y_val_preds = model.predict(X_val)
            mae = mean_absolute_error(y_val, y_val_preds)
            r2 = r2_score(y_val, y_val_preds)

            print(f"FOLD {fold} | MAE: {mae:.5f} | R2: {r2:.5f}")
            mae_per_fold.append(mae)
            r2_per_fold.append(r2)

            # Select the model with the lowest MAE
            if mae < self.best_mae:
                self.best_mae = mae
                self.best_model = model

        print(f"Average MAE across all folds: {np.mean(mae_per_fold):.5f}")
        print(f"Average R2 across all folds: {np.mean(r2_per_fold):.5f}")

    def predict(self):
        if self.best_model is None:
            raise ValueError("Model has not been trained yet. Call 'fit()' method first.")

        all_mae = []
        all_r2 = []
        y_preds_list = []
        y_val_list = []

        for fold, (X_train, X_val, y_train, y_val) in enumerate(self.split_data()):
            y_preds = self.best_model.predict(X_val)
            mae = mean_absolute_error(y_val, y_preds)
            r2 = r2_score(y_val, y_preds)
            all_mae.append(mae)
            all_r2.append(r2)
            y_preds_list.append(y_preds)
            y_val_list.append(y_val)

            print(f"FOLD {fold} | MAE: {mae:.5f} | R2: {r2:.5f}")

        avg_mae = np.mean(all_mae)
        avg_r2 = np.mean(all_r2)
        print(f'Average MAE Val: {avg_mae:.5f} | Average R2 Val: {avg_r2:.5f}')
        return all_mae, all_r2, y_preds_list, y_val_list

    def feature_importance(self):
        if self.best_model is None:
            raise ValueError("Model has not been trained yet. Call 'fit()' method first.")
        feature_importances = self.best_model.feature_importances_
        n_original_features = len(self.x.columns)
        polynomial_feature_indices = np.arange(n_original_features, self.x.shape[1])
        polynomial_feature_names = [f"Poly_{i}" for i in polynomial_feature_indices]
        all_feature_names = list(self.x.columns) + polynomial_feature_names
        feature_importances_df = pd.data({'feature_weight': feature_importances}, index=all_feature_names)
        feature_importances_df = feature_importances_df.sort_values('feature_weight', ascending=False)
        return feature_importances_df

    def inverse(self, y_preds, y_val):
        y_preds = np.expm1(y_preds)
        y_val = np.expm1(y_val)
        return y_preds, y_val


In [304]:
    def split_data(self):
        cv = RepeatedKFold(n_splits=self.n_splits, n_repeats=self.repeated, random_state=self.rdm_seed)
        for fold, (train_idx, val_idx) in enumerate(cv.split(self.x, self.y)):
            X_train = self.x.iloc[train_idx]
            X_val = self.x.iloc[val_idx]
            y_train = self.y.iloc[train_idx]
            y_val = self.y.iloc[val_idx]
            yield X_train, X_val, y_train, y_val


In [305]:
    def fit(self, key):
        if key == 'xgb':
            self._fit_xgb()
        elif key == 'lgb':
            self._fit_lgb()
        elif key == 'cat':
            self._fit_cat()



In [306]:
def _fit_xgb(self):
    import xgboost as xgb
    mae_per_fold = []
    r2_per_fold = []
    self.cv_splitter = self.split_data()

    for fold, (X_train, X_val, y_train, y_val) in enumerate(self.cv_splitter):
        model = xgb.XGBRegressor(**self.params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='mae', early_stopping_rounds=100, verbose=False)
        eval_results = model.evals_result()
        val_mae = eval_results['validation_0']['mae'][-1]
        
        y_val_preds = model.predict(X_val)
        val_r2 = r2_score(y_val, y_val_preds)

        if val_mae < self.best_mae:
            self.best_mae = val_mae
            self.best_model = model

        print(f"FOLD {fold} | MAE: {val_mae:.5f} | R2: {val_r2:.5f}")
        mae_per_fold.append(val_mae)
        r2_per_fold.append(val_r2)

    print(f"Average MAE across all folds: {np.mean(mae_per_fold):.5f}")
    print(f"Average R2 across all folds: {np.mean(r2_per_fold):.5f}")

In [307]:
    def _fit_adaboost(self, X_train, y_train):
        from sklearn.ensemble import AdaBoostRegressor
        base_estimator = DecisionTreeRegressor(max_depth=1)
        adaboost = AdaBoostRegressor(base_estimator=base_estimator, **self.params)
        adaboost.fit(X_train, y_train)
        return adaboost

In [308]:
def _fit_lgb(self):
    import lightgbm as lgb
    mae_per_fold = []
    r2_per_fold = []
    self.cv_splitter = self.split_data()

    for fold, (X_train, X_val, y_train, y_val) in enumerate(self.cv_splitter):
        model = lgb.LGBMRegressor(**self.params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='mae', early_stopping_rounds=100)
        eval_results = model.evals_result_
        val_mae = eval_results['valid_0']['l1'][-1]
        
        y_val_preds = model.predict(X_val)
        val_r2 = r2_score(y_val, y_val_preds)

        if val_mae < self.best_mae:
            self.best_mae = val_mae
            self.best_model = model

        print(f"FOLD {fold} | MAE: {val_mae:.5f} | R2: {val_r2:.5f}")
        mae_per_fold.append(val_mae)
        r2_per_fold.append(val_r2)

    print(f"Average MAE across all folds: {np.mean(mae_per_fold):.5f}")
    print(f"Average R2 across all folds: {np.mean(r2_per_fold):.5f}")

In [309]:
def _fit_cat(self):
    from catboost import CatBoostRegressor
    from sklearn.metrics import mean_absolute_error, r2_score
    rmse_per_fold = []
    mae_per_fold = []
    r2_per_fold = []
    self.cv_splitter = self.split_data()

    for fold, (X_train, X_val, y_train, y_val) in enumerate(self.cv_splitter):
        model = CatBoostRegressor(**self.params, loss_function='MAE')
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        y_pred = model.predict(X_val)

        val_mae = mean_absolute_error(y_val, y_pred)
        val_r2 = r2_score(y_val, y_pred)

        print(f"FOLD {fold} | MAE: {val_mae:.5f} | R²: {val_r2:.5f}")
        mae_per_fold.append(val_mae)
        r2_per_fold.append(val_r2)

        if val_mae < self.best_mae:
            self.best_mae = val_mae
            self.best_model = model

    print(f"Average MAE across all folds: {np.mean(mae_per_fold):.5f}")
    print(f"Average R² across all folds: {np.mean(r2_per_fold):.5f}")

In [310]:
    def predict(self):
        self.cv_splitter = self.split_data()
        all_rmse = []
        all_rmse_train = []
        for fold, (X_train, X_val, y_train, y_val) in enumerate(self.cv_splitter):
            if self.best_model is None:
                raise ValueError("Model has not been trained yet. Call 'fit()' method first.")
            else:
                y_preds = self.best_model.predict(X_val)
                rmse = np.sqrt(mean_squared_error(y_val, y_preds))
                all_rmse.append(rmse)

                train_preds = self.best_model.predict(X_train)
                train_rmse = np.sqrt(mean_squared_error(y_train, train_preds))
                all_rmse_train.append(train_rmse)
                print(f"FOLD {fold} | RMSE: {rmse:.5f}")
        avg_rmse = np.mean(all_rmse)
        avg_rmse_train = np.mean(all_rmse_train)
        print(f'Average RMSE Val: {avg_rmse} | Average RMSE Train: {avg_rmse_train}')
        return all_rmse, y_preds, y_val


In [ ]:
params = {'random_state':42}


model_xgb = Model(X_train_poly, y, 10, 3, 42, XGBRegressor, params)

# Fit the model using RepeatedKFold
model_xgb.fit('xgb')
all_mae_xgb,all_r2_xgb, y_preds, y_val = model_xgb.predict()

FOLD 0 | MAE: 71428.48500 | R2: 0.96747
FOLD 1 | MAE: 59676.04688 | R2: 0.97797
FOLD 2 | MAE: 54278.06443 | R2: 0.99166
FOLD 3 | MAE: 45619.28618 | R2: 0.99487
FOLD 4 | MAE: 65483.28574 | R2: 0.96591
FOLD 5 | MAE: 53549.22492 | R2: 0.98249
FOLD 6 | MAE: 49829.92311 | R2: 0.98716
FOLD 7 | MAE: 51529.58005 | R2: 0.97781
FOLD 8 | MAE: 53675.80930 | R2: 0.98768
FOLD 9 | MAE: 70610.68048 | R2: 0.97649
FOLD 10 | MAE: 56705.37527 | R2: 0.98024
FOLD 11 | MAE: 48560.97585 | R2: 0.99272
FOLD 12 | MAE: 68328.47018 | R2: 0.97429
FOLD 13 | MAE: 59247.27064 | R2: 0.98198
FOLD 14 | MAE: 40350.62193 | R2: 0.99492
FOLD 15 | MAE: 50983.97593 | R2: 0.98708
FOLD 16 | MAE: 50738.22682 | R2: 0.96572
FOLD 17 | MAE: 58434.94673 | R2: 0.98739
FOLD 18 | MAE: 62794.92577 | R2: 0.97329
FOLD 19 | MAE: 63256.30712 | R2: 0.98416
FOLD 20 | MAE: 42979.08638 | R2: 0.99308
FOLD 21 | MAE: 51019.87291 | R2: 0.99076
FOLD 22 | MAE: 64968.91972 | R2: 0.97575
FOLD 23 | MAE: 55243.13909 | R2: 0.98182
FOLD 24 | MAE: 55123.02248

In [ ]:
params = {'random_state':42,'verbosity':-1}
model_lgbm = Model(X_train_poly, y, 10, 3, 42, lgbm.LGBMRegressor, params)

# Fit the model using RepeatedKFold
model_lgbm.fit('lgb')
all_mae_lgb,all_r2_lgb, y_preds, y_val = model_lgbm.predict()

FOLD 0 | MAE: 79054.06356 | R2: 0.96209
FOLD 1 | MAE: 63533.17923 | R2: 0.98314
FOLD 2 | MAE: 62092.51269 | R2: 0.98445
FOLD 3 | MAE: 48123.87874 | R2: 0.99771
FOLD 4 | MAE: 72689.33124 | R2: 0.96522
FOLD 5 | MAE: 66251.69546 | R2: 0.98505
FOLD 6 | MAE: 61482.56201 | R2: 0.99055
FOLD 7 | MAE: 56029.05038 | R2: 0.98277
FOLD 8 | MAE: 64752.38388 | R2: 0.98212
FOLD 9 | MAE: 67678.98272 | R2: 0.97748
FOLD 10 | MAE: 68267.04966 | R2: 0.98128
FOLD 11 | MAE: 54555.19709 | R2: 0.98964
FOLD 12 | MAE: 77667.47412 | R2: 0.97183
FOLD 13 | MAE: 59614.06467 | R2: 0.99139
FOLD 14 | MAE: 53518.06385 | R2: 0.99108
FOLD 15 | MAE: 59273.24019 | R2: 0.98729
FOLD 16 | MAE: 60914.34474 | R2: 0.98344
FOLD 17 | MAE: 65162.76073 | R2: 0.98417
FOLD 18 | MAE: 72684.52595 | R2: 0.95539
FOLD 19 | MAE: 73167.15027 | R2: 0.96906
FOLD 20 | MAE: 56421.32996 | R2: 0.99245
FOLD 21 | MAE: 58122.79455 | R2: 0.99274
FOLD 22 | MAE: 73753.93095 | R2: 0.97861
FOLD 23 | MAE: 63042.60718 | R2: 0.97262
FOLD 24 | MAE: 61755.64805

In [312]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
# Parameter untuk Ridge Regression
params_ridge = {'alpha': 1.0}
model_ridge = Model(X_train_poly, y, 10, 3, 42, Ridge, params_ridge)
model_ridge.fit('ridge')
all_mae_ridge,all_r2_ridge, y_preds_ridge, y_val_ridge = model_ridge.predict()

FOLD 0 | MAE: 176538.20930 | R2: 0.99324
FOLD 1 | MAE: 172800.76098 | R2: 0.99448
FOLD 2 | MAE: 173914.41989 | R2: 0.99377
FOLD 3 | MAE: 169703.85508 | R2: 0.99209
FOLD 4 | MAE: 173416.75031 | R2: 0.98456
FOLD 5 | MAE: 177742.71049 | R2: 0.99263
FOLD 6 | MAE: 173259.08003 | R2: 0.99162
FOLD 7 | MAE: 171206.24896 | R2: 0.99238
FOLD 8 | MAE: 171202.47608 | R2: 0.99324
FOLD 9 | MAE: 176809.90784 | R2: 0.98357
FOLD 10 | MAE: 169780.66428 | R2: 0.99060
FOLD 11 | MAE: 172056.43175 | R2: 0.98959
FOLD 12 | MAE: 179680.78138 | R2: 0.99221
FOLD 13 | MAE: 170105.97204 | R2: 0.99429
FOLD 14 | MAE: 178623.72346 | R2: 0.98947
FOLD 15 | MAE: 173550.32760 | R2: 0.99188
FOLD 16 | MAE: 168488.12522 | R2: 0.99409
FOLD 17 | MAE: 175635.16511 | R2: 0.98351
FOLD 18 | MAE: 171323.66477 | R2: 0.98766
FOLD 19 | MAE: 186171.81214 | R2: 0.96578
FOLD 20 | MAE: 176978.48481 | R2: 0.98738
FOLD 21 | MAE: 166710.44271 | R2: 0.99461
FOLD 22 | MAE: 171230.52035 | R2: 0.99396
FOLD 23 | MAE: 172293.58259 | R2: 0.99359
FO

In [313]:
# Parameter untuk Linear Regression
params_linear = {}
model_linear = Model(X_train_poly, y, 10, 3, 42, LinearRegression, params_linear)
model_linear.fit('linear')
all_mae_linear,all_r2_linier, y_preds_linear, y_val_linear = model_linear.predict()

FOLD 0 | MAE: 180652.82031 | R2: 0.99293
FOLD 1 | MAE: 180680.58497 | R2: 0.99421
FOLD 2 | MAE: 177019.58009 | R2: 0.99361
FOLD 3 | MAE: 175182.76050 | R2: 0.99186
FOLD 4 | MAE: 173370.55872 | R2: 0.98578
FOLD 5 | MAE: 181252.91956 | R2: 0.99249
FOLD 6 | MAE: 176713.39064 | R2: 0.99156
FOLD 7 | MAE: 176468.58701 | R2: 0.99227
FOLD 8 | MAE: 177983.69970 | R2: 0.99323
FOLD 9 | MAE: 179186.39652 | R2: 0.98815
FOLD 10 | MAE: 173725.37360 | R2: 0.99332
FOLD 11 | MAE: 173188.40643 | R2: 0.98963
FOLD 12 | MAE: 182404.68951 | R2: 0.99294
FOLD 13 | MAE: 172976.54604 | R2: 0.99419
FOLD 14 | MAE: 182437.60196 | R2: 0.98936
FOLD 15 | MAE: 176489.86842 | R2: 0.99177
FOLD 16 | MAE: 172831.97492 | R2: 0.99398
FOLD 17 | MAE: 190064.65760 | R2: 0.92903
FOLD 18 | MAE: 177284.76790 | R2: 0.98742
FOLD 19 | MAE: 183915.71975 | R2: 0.98811
FOLD 20 | MAE: 181246.45130 | R2: 0.98733
FOLD 21 | MAE: 169551.74826 | R2: 0.99451
FOLD 22 | MAE: 176954.85693 | R2: 0.99385
FOLD 23 | MAE: 174455.64755 | R2: 0.99331
FO